In [1]:
!pip3 install scikit-image

In [1]:
from PIL import Image
import numpy as np
import pandas as pd
from skimage import io

# npys with image paths ex: array(['/../../imgpath1.jpg', '/../../imgpath2.jpg',...])
mit_path = np.load("MIT_Indoors_samples_paths_complete.npy")
sun_path = np.load("SUN397_Indoors_samples_paths_complete.npy")
img_size = 128

In [3]:
def img_reshape(img):
    img = Image.open(img).convert('RGB')
    img = img.resize((img_size, img_size))
    img = np.asarray(img)
    return img

def img_list_to_df(img_list: list, nome_imagem: list):
    num_images= len(img_list)
    img_arr_1d = []
    for c in range(num_images):
        # image 2d to 1d
        img_arr_1d.append(img_list[c].flatten())
    print("Num images 1d: ", len(img_arr_1d))
    print("Image 1d shape: ", img_arr_1d[0].shape)
    x_img = []

    # array 1d to list
    for c in range(num_images):
        x_img.append(img_arr_1d[c].tolist()) 

    # df with 1d images
    tam_colum = len(img_arr_1d[0])
    nome_colum = []
    for c in range(tam_colum):
        colum = 'pixel ' + str(c)
        nome_colum.append(colum)
    df = pd.DataFrame(x_img, columns= nome_colum)
    df.insert(loc = 0, column = 'image', value = nome_imagem)
    return df

def df_to_img_mosaic(df, save_img_name):
    x_data = df.iloc[0:,1:] #remove image name column

    #converte dataframe para vetor
    x_data = x_data.to_numpy()
    # calculate how many rows and columns we need (round up)
    cols = int(np.ceil(np.sqrt(x_data.shape[0])))
    rows = int(np.ceil(x_data.shape[0] * 1. / cols))
    im_w = im_h = img_size
    num_images = x_data.shape[0]

    X = np.reshape(x_data, (num_images, im_h, im_w, 3))

    if cols != rows:
        dif = cols - rows
        if cols > rows:
            rows = int(np.ceil((x_data.shape[0] * 1. / cols)+ dif))
        else:
            cols = int(np.ceil(np.sqrt((x_data.shape[0])+ dif))) 
    mosaic = np.zeros((rows * im_h, cols * im_w,3), dtype=np.uint8)

    for row in range(rows):
        for col in range(cols):
            im_index = col * rows + row
            if im_index < num_images:
                mosaic[col * im_h:(col+1) * im_h, row * im_w:(row+1) * im_w] = X[im_index, :, :]

    # write mosaic image to disk
    io.imsave(save_img_name, mosaic)

In [4]:
# MIT dataset
img_names = []
img_arr = []
for image in mit_path:
    img_arr.append(img_reshape(image))
    img_names.append(image)
print("Image shapes: ", img_arr[0].shape)

df = img_list_to_df(img_arr, img_names)
df_to_img_mosaic(df, "dataset_mit_mosaic.png")

Image shapes:  (128, 128, 3)
Num images 1d:  2535
Image 1d shape:  (49152,)


In [5]:
# SUN dataset
img_names = []
img_arr = []
for image in sun_path:
    img_arr.append(img_reshape(image))
    img_names.append(image)
print("Image shapes: ", img_arr[0].shape)

df = img_list_to_df(img_arr, img_names)
df_to_img_mosaic(df, "dataset_sun_mosaic.png")

Image shapes:  (128, 128, 3)
Num images 1d:  9285
Image 1d shape:  (49152,)
